In [1]:
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('DataFrames').getOrCreate()


24/04/04 19:14:31 WARN Utils: Your hostname, Koyanas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.29.17 instead (on interface en0)
24/04/04 19:14:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/04 19:14:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/04 19:14:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
fr=spark.read.csv('test.csv',header=True,inferSchema=True)
fr.show()

+------+----+------+----------+
|  Name| Age|Salary|Experience|
+------+----+------+----------+
| Rahul|  19| 15000|         0|
|Shweta|  20| 18000|      NULL|
| Mohit|  22| 16000|      NULL|
|  Raju|  25|  NULL|         1|
|Preeti|  30| 25000|      NULL|
|Laxman|  24| 32000|         2|
|Keshav|NULL|  NULL|      NULL|
+------+----+------+----------+



In [10]:
fr.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [11]:
fr.na.drop().show()

+------+---+------+----------+
|  Name|Age|Salary|Experience|
+------+---+------+----------+
| Rahul| 19| 15000|         0|
|Laxman| 24| 32000|         2|
+------+---+------+----------+



In [12]:
fr.na.drop(how="all").show()

+------+----+------+----------+
|  Name| Age|Salary|Experience|
+------+----+------+----------+
| Rahul|  19| 15000|         0|
|Shweta|  20| 18000|      NULL|
| Mohit|  22| 16000|      NULL|
|  Raju|  25|  NULL|         1|
|Preeti|  30| 25000|      NULL|
|Laxman|  24| 32000|         2|
|Keshav|NULL|  NULL|      NULL|
+------+----+------+----------+



In [13]:
fr.na.drop(how="any").show()

+------+---+------+----------+
|  Name|Age|Salary|Experience|
+------+---+------+----------+
| Rahul| 19| 15000|         0|
|Laxman| 24| 32000|         2|
+------+---+------+----------+



In [14]:
fr.na.drop(how="any",thresh=2).show()

+------+---+------+----------+
|  Name|Age|Salary|Experience|
+------+---+------+----------+
| Rahul| 19| 15000|         0|
|Shweta| 20| 18000|      NULL|
| Mohit| 22| 16000|      NULL|
|  Raju| 25|  NULL|         1|
|Preeti| 30| 25000|      NULL|
|Laxman| 24| 32000|         2|
+------+---+------+----------+



In [15]:
fr.na.drop(how="any",subset=['Experience']).show()

+------+---+------+----------+
|  Name|Age|Salary|Experience|
+------+---+------+----------+
| Rahul| 19| 15000|         0|
|  Raju| 25|  NULL|         1|
|Laxman| 24| 32000|         2|
+------+---+------+----------+



In [22]:
# Read the CSV file into a DataFrame
df = spark.read.option("header", "true").csv("test.csv")

# Fill NULL values with 'text' for all columns
df_filled = df.na.fill("text")

# Show the DataFrame
df_filled.show()


+------+----+------+----------+
|  Name| Age|Salary|Experience|
+------+----+------+----------+
| Rahul|  19| 15000|         0|
|Shweta|  20| 18000|      text|
| Mohit|  22| 16000|      text|
|  Raju|  25|  text|         1|
|Preeti|  30| 25000|      text|
|Laxman|  24| 32000|         2|
|Keshav|text|  text|      text|
+------+----+------+----------+



In [23]:
fr.show()
fr.printSchema()

+------+----+------+----------+
|  Name| Age|Salary|Experience|
+------+----+------+----------+
| Rahul|  19| 15000|         0|
|Shweta|  20| 18000|      NULL|
| Mohit|  22| 16000|      NULL|
|  Raju|  25|  NULL|         1|
|Preeti|  30| 25000|      NULL|
|Laxman|  24| 32000|         2|
|Keshav|NULL|  NULL|      NULL|
+------+----+------+----------+

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Experience: integer (nullable = true)



In [31]:
from pyspark.ml.feature import Imputer

# Specify the output columns for imputed values
output_cols = ["{}_imputed".format(c) for c in ['Age','Salary','Experience']]

try:
    imputer = Imputer(
        inputCols=['Age','Salary','Experience'],
        outputCols=output_cols
    ).setStrategy("mean")

    # Fit and transform the DataFrame
    imputed_df = imputer.fit(fr).transform(fr)

    # Show the DataFrame with imputed values
    imputed_df.show()

except Exception as e:
    print("An error occurred:", e)


+------+----+------+----------+-----------+--------------+------------------+
|  Name| Age|Salary|Experience|Age_imputed|Salary_imputed|Experience_imputed|
+------+----+------+----------+-----------+--------------+------------------+
| Rahul|  19| 15000|         0|         19|         15000|                 0|
|Shweta|  20| 18000|      NULL|         20|         18000|                 1|
| Mohit|  22| 16000|      NULL|         22|         16000|                 1|
|  Raju|  25|  NULL|         1|         25|         21200|                 1|
|Preeti|  30| 25000|      NULL|         30|         25000|                 1|
|Laxman|  24| 32000|         2|         24|         32000|                 2|
|Keshav|NULL|  NULL|      NULL|         23|         21200|                 1|
+------+----+------+----------+-----------+--------------+------------------+



In [32]:
fr.filter("Salary <= 20000").show()

+------+---+------+----------+
|  Name|Age|Salary|Experience|
+------+---+------+----------+
| Rahul| 19| 15000|         0|
|Shweta| 20| 18000|      NULL|
| Mohit| 22| 16000|      NULL|
+------+---+------+----------+



In [33]:
fr.filter("Salary <= 20000").select(['Name','Age']).show()

+------+---+
|  Name|Age|
+------+---+
| Rahul| 19|
|Shweta| 20|
| Mohit| 22|
+------+---+



In [35]:
fr.filter((fr['Salary']<=25000) & (fr['Salary']>=20000)).show()

+------+---+------+----------+
|  Name|Age|Salary|Experience|
+------+---+------+----------+
|Preeti| 30| 25000|      NULL|
+------+---+------+----------+



In [36]:
fr.filter(~(fr['Salary']<=20000)).show()

+------+---+------+----------+
|  Name|Age|Salary|Experience|
+------+---+------+----------+
|Preeti| 30| 25000|      NULL|
|Laxman| 24| 32000|         2|
+------+---+------+----------+

